# NLP 과제
- 한국어 데이터셋으로 전처리

   : 정규표현식, 불용어제거, mecab 토큰화, 워드임베딩

- 하이퍼파라미터

 : batch size 32, lr 0.001, epochs 40

- 모델

 : dropout 비율:  0.2 / 0.4 변경


RNN 정확도:  0.318

GRU 정확도:  0.343

# 한국어 문장 관계 분류

├ Index : train data index

├ Premise : 실제 Text

├ Hypothesis : 가설 Text

└ Label : 참(Entailment) 또는 거짓(Contradiction) 또는 중립(Neutral)

https://dacon.io/competitions/official/235875/overview/description

## 라이브러리 import/기본 설정

In [1]:
import os
import torch 
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import random

import pandas as pd
import re

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

## 데이터 불러오기

In [40]:
train = pd.read_csv("/content/drive/MyDrive/한국어 NLI/train_data.csv")
test = pd.read_csv("/content/drive/MyDrive/한국어 NLI/test_data.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/한국어 NLI/sample_submission.csv")

In [41]:
train.head()

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral


In [42]:
test.head()

,index,premise,hypothesis,label
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,answer
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,answer
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,answer
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,answer
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,answer


In [43]:
sample_submission.head()

,index,label
0,0,answer
1,1,answer
2,2,answer
3,3,answer
4,4,answer


## 전처리 
**1. 정규표현식**

https://wikidocs.net/4308 에서 다양한 식을 소개하고 있으니 참고

In [44]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 0-9 ]', '', text)

In [45]:
train['premise']=train['premise'].apply(alpha_num)
train['hypothesis']=train['hypothesis'].apply(alpha_num)

test['premise']=test['premise'].apply(alpha_num)
test['hypothesis']=test['hypothesis'].apply(alpha_num)

In [46]:
test.iloc[0,1] # 문장 부호가 모두 없어진 것을 확인할 수 있음

'다만 조금 좁아서 케리어를 펼치기 불편합니다'

**2. 불용어 제거하기**

한글 불용어를 정리해 놓은 링크 https://www.ranks.nl/stopwords/korean

In [47]:
# 영어 불용어 제거하기
import nltk
from nltk.corpus import stopwords 

nltk.download('stopwords')
stopwords = stopwords.words('english')

print(stopwords[:10])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [48]:
# 한국어 불용어 제거하기
import nltk
from nltk.corpus import stopwords 

stopwords = ['아', '휴', '아이구', '아이쿠', '아이고', '어'] # 예측 목적에 도움이 되지 않을 것 같은 불용어 제거

**3. 토큰화: 텍스트를 쪼개는 것**
- 문장
- 단어
- 형태소

다양한 기준으로 텍스트를 쪼갤 수 있으니 각자의 목적에 맞게 고를 것
특히 한글의 경우 단어로 쪼갤 것인지, 형태소로 쪼갤 것인지에 대한 고민 필요

In [14]:
!pip install konlpy
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     |████████████████████████████████| 19.4 MB 4.2 MB/s 
     |████████████████████████████████| 448 kB 57.3 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.16).
git is already the newest version (1:2.17.1-1ubuntu0.9).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-mem

In [ ]:
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

text = "대보름 날에 다들 소원 비셨나요?"
tokenizer.tokenize(text)

['대보름', '날에', '다들', '소원', '비셨나요', '?']

In [49]:
from konlpy.tag import Mecab

text = "대보름 날에 다들 소원 비셨나요?"
tokenizer = Mecab()
print(tokenizer.morphs(text))

['대', '보름', '날', '에', '다', '들', '소원', '비', '셨', '나요', '?']


In [50]:
# mecab, 불용어 제거
def remove_stopwords(text):
    final_text = []
    words = tokenizer.morphs(text)
    for word in words:
        if word.strip().lower() not in stopwords:
          final_text.append(word.strip())
    return  " ".join(final_text)

In [51]:
# train['premise'] = train['premise'].str.lower()
# test['premise'] = test['premise'].str.lower()
train['premise'] = train['premise'].apply(alpha_num).apply(remove_stopwords)
test['premise'] = test['premise'].apply(alpha_num).apply(remove_stopwords)

In [52]:
# train['hypothesis'] = train['hypothesis'].str.lower()
# test['hypothesis'] = test['hypothesis'].str.lower()
train['hypothesis'] = train['hypothesis'].apply(alpha_num).apply(remove_stopwords)
test['hypothesis'] = test['hypothesis'].apply(alpha_num).apply(remove_stopwords)

In [53]:
train.head()

,index,premise,hypothesis,label
0,0,씨름 은 상고 시대 로부터 전해져 내려오 는 남자 들 의 대표 적 인 놀이 로서 소...,씨름 의 여자 들 의 놀이 이 다,contradiction
1,1,삼성 은 자작극 을 벌인 2 명 에게 형사 고소 등 의 법 적 대응 을 검토 중 이...,자작극 을 벌인 이 는 3 명 이 다,contradiction
2,2,이 를 위해 예측 적 범죄 예방 시스템 을 구축 하 고 고도 화 한다,예측 적 범죄 예방 시스템 구축 하 고 고도 화 하 는 것 은 목적 이 있 기 때문...,entailment
3,3,광주 광역시 가 재 개발 정비 사업 원주민 들 에 대한 종합 대책 을 마련 하 는 ...,원주민 들 은 종합 대책 에 만족 했 다,neutral
4,4,진정 소비자 와 직원 들 에게 사랑 받 는 기업 으로 오래 지속 되 고 싶 으면 이...,이런 상황 에서 책임 있 는 모습 을 보여 주 는 기업 은 아주 드물 다,neutral


In [54]:
test.head()

,index,premise,hypothesis,label
0,0,다만 조금 좁 아서 케리어 를 펼치 기 불편 합니다,케리어 를 펼치 기 에 공간 이 충분 했 습니다,answer
1,1,그리고 위치 가 시 먼 역 보다 는 샤오난 먼 역 에 가까워 요,시먼 역 보다 는 샤오난 먼 역 에 먼저 도착 할 수 있 어요,answer
2,2,구구절절 설명 하 고 이해 시키 려는 노력 이 큰 의미 없이 다가온다,무엇 인가 말 을 많이 하 기 는 했 지만 큰 의미 가 있 지 는 않 았 다,answer
3,3,몇 번 을 다시 봐도 볼 때 마다 가슴 이 저민다,다시 봤 을 때 는 무덤덤 했 다,answer
4,4,8 월 중 에 입주 신청 을 하 면 청년 은 9 월 신혼 부부 는 10 월 부터 입...,8 월 중 에 입주 신청 을 하 면 신혼 부부 는 9 월 부터 입주 가 가능 하 다,answer


In [55]:
train["text"] = train["premise"] + "[SEP]" + train["hypothesis"]
test["text"] = test["premise"] + "[SEP]" + test["hypothesis"]

In [56]:
train.head()

,index,premise,hypothesis,label,text
0,0,씨름 은 상고 시대 로부터 전해져 내려오 는 남자 들 의 대표 적 인 놀이 로서 소...,씨름 의 여자 들 의 놀이 이 다,contradiction,씨름 은 상고 시대 로부터 전해져 내려오 는 남자 들 의 대표 적 인 놀이 로서 소...
1,1,삼성 은 자작극 을 벌인 2 명 에게 형사 고소 등 의 법 적 대응 을 검토 중 이...,자작극 을 벌인 이 는 3 명 이 다,contradiction,삼성 은 자작극 을 벌인 2 명 에게 형사 고소 등 의 법 적 대응 을 검토 중 이...
2,2,이 를 위해 예측 적 범죄 예방 시스템 을 구축 하 고 고도 화 한다,예측 적 범죄 예방 시스템 구축 하 고 고도 화 하 는 것 은 목적 이 있 기 때문...,entailment,이 를 위해 예측 적 범죄 예방 시스템 을 구축 하 고 고도 화 한다[SEP]예측 ...
3,3,광주 광역시 가 재 개발 정비 사업 원주민 들 에 대한 종합 대책 을 마련 하 는 ...,원주민 들 은 종합 대책 에 만족 했 다,neutral,광주 광역시 가 재 개발 정비 사업 원주민 들 에 대한 종합 대책 을 마련 하 는 ...
4,4,진정 소비자 와 직원 들 에게 사랑 받 는 기업 으로 오래 지속 되 고 싶 으면 이...,이런 상황 에서 책임 있 는 모습 을 보여 주 는 기업 은 아주 드물 다,neutral,진정 소비자 와 직원 들 에게 사랑 받 는 기업 으로 오래 지속 되 고 싶 으면 이...


In [57]:
test.head()

,index,premise,hypothesis,label,text
0,0,다만 조금 좁 아서 케리어 를 펼치 기 불편 합니다,케리어 를 펼치 기 에 공간 이 충분 했 습니다,answer,다만 조금 좁 아서 케리어 를 펼치 기 불편 합니다[SEP]케리어 를 펼치 기 에 ...
1,1,그리고 위치 가 시 먼 역 보다 는 샤오난 먼 역 에 가까워 요,시먼 역 보다 는 샤오난 먼 역 에 먼저 도착 할 수 있 어요,answer,그리고 위치 가 시 먼 역 보다 는 샤오난 먼 역 에 가까워 요[SEP]시먼 역 보...
2,2,구구절절 설명 하 고 이해 시키 려는 노력 이 큰 의미 없이 다가온다,무엇 인가 말 을 많이 하 기 는 했 지만 큰 의미 가 있 지 는 않 았 다,answer,구구절절 설명 하 고 이해 시키 려는 노력 이 큰 의미 없이 다가온다[SEP]무엇 ...
3,3,몇 번 을 다시 봐도 볼 때 마다 가슴 이 저민다,다시 봤 을 때 는 무덤덤 했 다,answer,몇 번 을 다시 봐도 볼 때 마다 가슴 이 저민다[SEP]다시 봤 을 때 는 무덤덤...
4,4,8 월 중 에 입주 신청 을 하 면 청년 은 9 월 신혼 부부 는 10 월 부터 입...,8 월 중 에 입주 신청 을 하 면 신혼 부부 는 9 월 부터 입주 가 가능 하 다,answer,8 월 중 에 입주 신청 을 하 면 청년 은 9 월 신혼 부부 는 10 월 부터 입...


**4-1. 카운터 기반 임베딩**

컴퓨터가 알아들을 수 있도록 문자를 벡터로 바꾸어줌

원핫 인코딩 간단한 방법, 그러나 매우 sparse 한 행렬 -> inefficient

카운트 기반 인코딩(BoG 등): tf-idf

여기서는 tf-idf 이용

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer(max_features = 128) # max_features를 통해 최대 몇 개의 단어를 벡터로 바꿀 것인지 결정합니다. 
v.fit(train['text']) # test 때는 train 에서 학습된 tf-idf를 이용해야 하기 때문에 여기서는 fit_transform을 한꺼번에 쓰는 대신, 나눠서 이용합니다 ~ sklearn에서 여타 

TfidfVectorizer(max_features=128)

In [59]:
print(v.vocabulary_) # 벡터화된 단어들 사전

{'대표': 31, '이나': 91, '에서': 80, 'sep': 2, '에게': 79, '라고': 42, '라는': 43, '위해': 89, '한다': 120, '때문': 40, '사업': 59, '대한': 32, '사랑': 57, '기업': 18, '으로': 90, '이런': 92, '상황': 64, '필요': 116, '이번': 93, '자신': 99, '아니': 75, '모두': 47, '모든': 48, '사진': 61, '습니다': 70, '깨끗': 20, '어요': 78, '다는': 26, '진행': 109, '된다': 37, '영화': 83, '시작': 73, '부터': 55, '까지': 19, '함께': 121, '정보': 103, '아직': 76, '가족': 5, '시간': 71, '동안': 36, '내용': 21, '계획': 12, '우리': 85, '10': 0, '지역': 107, '면서': 46, '시장': 74, '는다': 23, '보다': 54, '관리': 14, '위한': 88, '지원': 108, '정부': 104, '사람': 56, '경우': 9, '통해': 115, '운영': 86, '입니다': 98, '최고': 112, '행사': 124, '지난': 105, '확인': 126, '경제': 10, '한국': 119, '해야': 123, '학교': 118, '대상': 29, '다고': 25, '정말': 102, '다른': 27, '드라마': 38, '감동': 7, '사용': 60, '작품': 100, '는데': 24, '연기': 82, '정도': 101, '생각': 65, '이상': 94, '안전': 77, '추진': 113, '예정': 84, '따라': 39, '국가': 16, '관련': 13, '지만': 106, '일본': 97, '대통령': 30, '매우': 45, '친절': 114, '합니다': 122, '이용': 95, '시설': 72, '이후': 96, '가능': 3, '도록': 35, '거리': 8, '사회': 6

In [60]:
x = v.transform(train['text']).toarray()
print(x)

[[0.         0.         0.08098936 ... 0.         0.         0.        ]
 [0.         0.         0.11963594 ... 0.         0.         0.        ]
 [0.         0.         0.13968396 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.07364782 ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 [0.         0.         0.08629155 ... 0.         0.         0.        ]]


In [61]:
print(x.shape) # 128개의 max 단어를 설정했기 때문에 문장수, 단어수 이렇게 array를 만들어진 것을 확인할 수 있다
print(train.shape)

(24998, 128)
(24998, 5)


In [62]:
x = pd.DataFrame(x)
temp = []
for i in range(len(x)):
  temp.append(list(x.iloc[i,:]))
train['preprocessed_text'] = temp

In [65]:
train

,index,premise,hypothesis,label,text,preprocessed_text
0,0,씨름 은 상고 시대 로부터 전해져 내려오 는 남자 들 의 대표 적 인 놀이 로서 소...,씨름 의 여자 들 의 놀이 이 다,contradiction,씨름 은 상고 시대 로부터 전해져 내려오 는 남자 들 의 대표 적 인 놀이 로서 소...,"[0.0, 0.0, 0.08098936468958007, 0.0, 0.0, 0.0,..."
1,1,삼성 은 자작극 을 벌인 2 명 에게 형사 고소 등 의 법 적 대응 을 검토 중 이...,자작극 을 벌인 이 는 3 명 이 다,contradiction,삼성 은 자작극 을 벌인 2 명 에게 형사 고소 등 의 법 적 대응 을 검토 중 이...,"[0.0, 0.0, 0.11963594071371271, 0.0, 0.0, 0.0,..."
2,2,이 를 위해 예측 적 범죄 예방 시스템 을 구축 하 고 고도 화 한다,예측 적 범죄 예방 시스템 구축 하 고 고도 화 하 는 것 은 목적 이 있 기 때문...,entailment,이 를 위해 예측 적 범죄 예방 시스템 을 구축 하 고 고도 화 한다[SEP]예측 ...,"[0.0, 0.0, 0.13968395630466443, 0.0, 0.0, 0.0,..."
3,3,광주 광역시 가 재 개발 정비 사업 원주민 들 에 대한 종합 대책 을 마련 하 는 ...,원주민 들 은 종합 대책 에 만족 했 다,neutral,광주 광역시 가 재 개발 정비 사업 원주민 들 에 대한 종합 대책 을 마련 하 는 ...,"[0.0, 0.0, 0.1506107088000027, 0.0, 0.0, 0.0, ..."
4,4,진정 소비자 와 직원 들 에게 사랑 받 는 기업 으로 오래 지속 되 고 싶 으면 이...,이런 상황 에서 책임 있 는 모습 을 보여 주 는 기업 은 아주 드물 다,neutral,진정 소비자 와 직원 들 에게 사랑 받 는 기업 으로 오래 지속 되 고 싶 으면 이...,"[0.0, 0.0, 0.046577609106844255, 0.0, 0.0, 0.0..."
...,...,...,...,...,...,...
24993,24993,오페라 에 비하 여 오라토리오 에서 는 독창 보다 도 합창 이 중시 되 며 테스 토...,오라토리오 에서 테스 토의 역할 이 가장 중요 하 다,neutral,오페라 에 비하 여 오라토리오 에서 는 독창 보다 도 합창 이 중시 되 며 테스 토...,"[0.0, 0.0, 0.07617363801602103, 0.0, 0.3669034..."
24994,24994,지하철역 까지 걸어서 5 분 정도 걸립니다,지하철역 까지 도보 로 5 분 정도 걸립니다,entailment,지하철역 까지 걸어서 5 분 정도 걸립니다[SEP]지하철역 까지 도보 로 5 분 정...,"[0.0, 0.0, 0.07614360171428874, 0.0, 0.0, 0.0,..."
24995,24995,한편 이날 중앙 방역 대책 본부 는 집단 감염 이 발생 한 음식점 관련 역학 조사 ...,중악 방역 대책 본부 는 집단 감염 과 관련 한 모든 정보 를 비 공개 했 다,contradiction,한편 이날 중앙 방역 대책 본부 는 집단 감염 이 발생 한 음식점 관련 역학 조사 ...,"[0.0, 0.0, 0.07364781545198822, 0.0, 0.0, 0.0,..."
24996,24996,마미손 이 랩 을 하 자 시청자 들 은 그 의 정체 를 파악 했 다,시청자 들 은 마미손 의 정체 를 안다,entailment,마미손 이 랩 을 하 자 시청자 들 은 그 의 정체 를 파악 했 다[SEP]시청자 ...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


**4-2. 워드 임베딩**

위의 tf-idf 는 단순히 카운트를 기반으로 matrix를 만듭니다

하지만 단어와 단어 사이에서 그 의미가 달라질 수도 있습니다
이걸 고려하기 위해서 나온 또다른 벡터화/임배딩 방법이 워드 임배딩입니다

torch모델에서 임배딩도 학습하게 하는 구문이 있으므로 그걸 보면서 하도록 합시다.

In [66]:
word_set = []
max_len = 0

for d in train['text']:
  word_set = word_set + d.split(' ') # 여기에 토큰화한 데이터가 들어가면 됩니다
  if len(d.split()) > max_len:
    max_len = len(d.split())
  
word_set = set(word_set)

In [67]:
word_to_idx = {word: i+1 for i, word in enumerate(word_set)}
print(len(word_set))
print(max_len)

30142
88


In [68]:
def word_to_key(text):
  final_text = []
  for word in text.split():
      final_text.append(word_to_idx[word])
  if len(final_text) < max_len:
    final_text = final_text + [0] * (max_len - len(final_text))
  return final_text


train['word_to_key'] = train['text'].apply(word_to_key)

In [70]:
train.head()

,index,premise,hypothesis,label,text,preprocessed_text,word_to_key
0,0,씨름 은 상고 시대 로부터 전해져 내려오 는 남자 들 의 대표 적 인 놀이 로서 소...,씨름 의 여자 들 의 놀이 이 다,contradiction,씨름 은 상고 시대 로부터 전해져 내려오 는 남자 들 의 대표 적 인 놀이 로서 소...,"[0.0, 0.0, 0.08098936468958007, 0.0, 0.0, 0.0,...","[28846, 7681, 13774, 8694, 10327, 11134, 19136..."
1,1,삼성 은 자작극 을 벌인 2 명 에게 형사 고소 등 의 법 적 대응 을 검토 중 이...,자작극 을 벌인 이 는 3 명 이 다,contradiction,삼성 은 자작극 을 벌인 2 명 에게 형사 고소 등 의 법 적 대응 을 검토 중 이...,"[0.0, 0.0, 0.11963594071371271, 0.0, 0.0, 0.0,...","[17782, 7681, 19095, 28669, 19064, 20506, 1177..."
2,2,이 를 위해 예측 적 범죄 예방 시스템 을 구축 하 고 고도 화 한다,예측 적 범죄 예방 시스템 구축 하 고 고도 화 하 는 것 은 목적 이 있 기 때문...,entailment,이 를 위해 예측 적 범죄 예방 시스템 을 구축 하 고 고도 화 한다[SEP]예측 ...,"[0.0, 0.0, 0.13968395630466443, 0.0, 0.0, 0.0,...","[17319, 15051, 7343, 30142, 1175, 12632, 11803..."
3,3,광주 광역시 가 재 개발 정비 사업 원주민 들 에 대한 종합 대책 을 마련 하 는 ...,원주민 들 은 종합 대책 에 만족 했 다,neutral,광주 광역시 가 재 개발 정비 사업 원주민 들 에 대한 종합 대책 을 마련 하 는 ...,"[0.0, 0.0, 0.1506107088000027, 0.0, 0.0, 0.0, ...","[7467, 5631, 5411, 6320, 29711, 29128, 6967, 1..."
4,4,진정 소비자 와 직원 들 에게 사랑 받 는 기업 으로 오래 지속 되 고 싶 으면 이...,이런 상황 에서 책임 있 는 모습 을 보여 주 는 기업 은 아주 드물 다,neutral,진정 소비자 와 직원 들 에게 사랑 받 는 기업 으로 오래 지속 되 고 싶 으면 이...,"[0.0, 0.0, 0.046577609106844255, 0.0, 0.0, 0.0...","[27622, 20334, 29006, 4896, 11493, 14141, 2805..."


In [97]:
train['label'] = train['label'].replace(['contradiction','entailment','neutral'],[0,1,2])

In [98]:
train.head()

,index,premise,hypothesis,label,text,preprocessed_text,word_to_key
0,0,씨름 은 상고 시대 로부터 전해져 내려오 는 남자 들 의 대표 적 인 놀이 로서 소...,씨름 의 여자 들 의 놀이 이 다,0,씨름 은 상고 시대 로부터 전해져 내려오 는 남자 들 의 대표 적 인 놀이 로서 소...,"[0.0, 0.0, 0.08098936468958007, 0.0, 0.0, 0.0,...","[28846, 7681, 13774, 8694, 10327, 11134, 19136..."
1,1,삼성 은 자작극 을 벌인 2 명 에게 형사 고소 등 의 법 적 대응 을 검토 중 이...,자작극 을 벌인 이 는 3 명 이 다,0,삼성 은 자작극 을 벌인 2 명 에게 형사 고소 등 의 법 적 대응 을 검토 중 이...,"[0.0, 0.0, 0.11963594071371271, 0.0, 0.0, 0.0,...","[17782, 7681, 19095, 28669, 19064, 20506, 1177..."
2,2,이 를 위해 예측 적 범죄 예방 시스템 을 구축 하 고 고도 화 한다,예측 적 범죄 예방 시스템 구축 하 고 고도 화 하 는 것 은 목적 이 있 기 때문...,1,이 를 위해 예측 적 범죄 예방 시스템 을 구축 하 고 고도 화 한다[SEP]예측 ...,"[0.0, 0.0, 0.13968395630466443, 0.0, 0.0, 0.0,...","[17319, 15051, 7343, 30142, 1175, 12632, 11803..."
3,3,광주 광역시 가 재 개발 정비 사업 원주민 들 에 대한 종합 대책 을 마련 하 는 ...,원주민 들 은 종합 대책 에 만족 했 다,2,광주 광역시 가 재 개발 정비 사업 원주민 들 에 대한 종합 대책 을 마련 하 는 ...,"[0.0, 0.0, 0.1506107088000027, 0.0, 0.0, 0.0, ...","[7467, 5631, 5411, 6320, 29711, 29128, 6967, 1..."
4,4,진정 소비자 와 직원 들 에게 사랑 받 는 기업 으로 오래 지속 되 고 싶 으면 이...,이런 상황 에서 책임 있 는 모습 을 보여 주 는 기업 은 아주 드물 다,2,진정 소비자 와 직원 들 에게 사랑 받 는 기업 으로 오래 지속 되 고 싶 으면 이...,"[0.0, 0.0, 0.046577609106844255, 0.0, 0.0, 0.0...","[27622, 20334, 29006, 4896, 11493, 14141, 2805..."


# 모델

In [118]:
# 하이퍼파라미터
batch_size = 32
lr = 0.001
epochs = 40

In [72]:
# X = train.loc[:, "word_to_key"]
# y = train.loc[: , "label"]
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state= 33)

In [86]:
X_train.shape

(18748,)

In [119]:
X_train = train.iloc[:18748, 6]
X_test = train.iloc[18748:, 6].reset_index(drop=True) # 인덱스를 다시 처음부터 재배열 해주는 함수

y_train = train.iloc[:18748, 3]
y_test = train.iloc[18748:, 3].reset_index(drop=True) 

In [120]:
class CustomDataset(Dataset):
  def __init__(self):
    
    self.x_data = X_train
    self.y_data = [[y] for y in y_train]

  def __len__(self):

    return len(self.x_data)

  def __getitem__(self, idx):

    x = torch.LongTensor(self.x_data[idx]).to(device)
    y = torch.LongTensor(self.y_data[idx]).to(device)

    return x,y

In [121]:
class CustomDataset_test(Dataset):
  def __init__(self):
    
    self.x_data = X_test
    self.y_data = [[y] for y in y_test]

  def __len__(self):

    return len(self.x_data)

  def __getitem__(self, idx):

    x = torch.LongTensor(self.x_data[idx])
    y = torch.LongTensor(self.y_data[idx])

    return x,y

In [122]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=batch_size) #sampler를 만들었다면, sampler를 파라미터로 넣어줄 수 있음 : https://hyelimkungkung.tistory.com/29?category=935193

## RNN 모델

In [123]:
# 여기서 혹시 gru를 써보고 싶다면 주석을 풀고 시행하면 됩니다. 

class RNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
      super(RNN, self).__init__()
      self.n_layers = n_layers
      self.hidden_dim = hidden_dim

      self.embed = nn.Embedding(n_vocab, embed_dim)
      self.dropout = nn.Dropout(dropout_p)
      # self.gru = nn.GRU(embed_dim, self.hidden_dim,
      #                  num_layers=self.n_layers,
      #                  batch_first=True)
      self.rnn = nn.RNN(embed_dim, self.hidden_dim,batch_first = True)
      self.out = nn.Sequential(
          nn.Linear(self.hidden_dim, n_classes),
          nn.Softmax()
      )
    def forward(self, x):
      x = self.embed(x)
      h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
      #x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
      x, _ = self.rnn(x,h_0)
      h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
      self.dropout(h_t)
      logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
      return logit

    def _init_state(self, batch_size=1):
      weight = next(self.parameters()).data
      return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [124]:
n_vocab = 30142+1
embedd_size = 5
hidden_size = 100
output_size = 3

In [125]:
net = RNN(1, 256, n_vocab, embedd_size, output_size, 0.5).to(device)

In [126]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr)

In [127]:
losses = []
for epoch in range(epochs):
  
  for x, y in dataloader:
    optimizer.zero_grad()
    y = y.to(device)

    # forward 연산
    hypothesis = net(x)

    # 비용 함수
    y = y.squeeze()
    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()
    losses.append(cost.item()) # 값만 가져오기 위해서 .item()

  # 10의 배수에 해당되는 에포크마다 비용을 출력
  if epoch % 10 == 0:
      print(epoch, cost.item())

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0 1.1053335666656494
10 1.1081082820892334
20 1.08573317527771
30 1.102541208267212


In [128]:
dataset = CustomDataset_test()
test_loader = DataLoader(dataset, batch_size=batch_size)

In [129]:
correct = 0

with torch.no_grad():
  net = net.to('cpu')
  net.eval()
  for data, target in test_loader:
    data, target = data, target
    output = net(data)
    
    pred = output.max(1, keepdim=True)[1]
    # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
    correct += pred.eq(target.view_as(pred)).sum().item()

test_accuracy = correct / len(test_loader.dataset)
print('Accuracy:', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Accuracy: 0.3184


In [130]:
pred

tensor([[2],
        [2],
        [2],
        [2],
        [1],
        [2],
        [0],
        [2],
        [2],
        [2]])

## GRU

## 모델1
dropout = 0.4

In [132]:
from pandas.core.apply import GroupByApply
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.4):
      super(GRU, self).__init__()
      self.n_layers = n_layers
      self.hidden_dim = hidden_dim

      self.embed = nn.Embedding(n_vocab, embed_dim)
      self.dropout = nn.Dropout(dropout_p)
      self.gru = nn.GRU(embed_dim, self.hidden_dim,
                        num_layers=self.n_layers,
                       batch_first=True)
      # self.rnn = nn.RNN(embed_dim, self.hidden_dim,batch_first = True)
      self.out = nn.Sequential(
          nn.Linear(self.hidden_dim, n_classes),
          nn.Softmax()
      )
    def forward(self, x):
      x = self.embed(x)
      h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
      x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
      # x, _ = self.rnn(x,h_0)
      h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
      self.dropout(h_t)
      logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
      return logit

    def _init_state(self, batch_size=1):
      weight = next(self.parameters()).data
      return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [133]:
net = GRU(1, 256, n_vocab, embedd_size, output_size, 0.5).to(device)

In [135]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr)

In [145]:
losses = []
for epoch in range(epochs):
  
  for x, y in dataloader:
    optimizer.zero_grad()
    y = y.to(device)

    # forward 연산
    hypothesis = net(x)

    # 비용 함수
    y = y.squeeze()
    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()
    losses.append(cost.item()) # 값만 가져오기 위해서 .item()

  # 10의 배수에 해당되는 에포크마다 비용을 출력
  if epoch % 10 == 0:
      print(epoch, cost.item())

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0 1.0998332500457764
10 1.0975710153579712
20 1.0914477109909058
30 1.090862512588501


In [137]:
dataset = CustomDataset_test()
test_loader = DataLoader(dataset, batch_size=batch_size)

In [147]:
correct = 0

with torch.no_grad():
  net = net.to('cpu')
  net.eval()
  for data, target in test_loader:
    data, target = data, target
    output = net(data)
    
    pred = output.max(1, keepdim=True)[1]
    # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
    correct += pred.eq(target.view_as(pred)).sum().item()

test_accuracy = correct / len(test_loader.dataset)
print('Accuracy:', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Accuracy: 0.25232


## 모델2
Dropout 비율 0.2

In [142]:
from pandas.core.apply import GroupByApply
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
      super(GRU, self).__init__()
      self.n_layers = n_layers
      self.hidden_dim = hidden_dim

      self.embed = nn.Embedding(n_vocab, embed_dim)
      self.dropout = nn.Dropout(dropout_p)
      self.gru = nn.GRU(embed_dim, self.hidden_dim,
                        num_layers=self.n_layers,
                       batch_first=True)
      # self.rnn = nn.RNN(embed_dim, self.hidden_dim,batch_first = True)
      self.out = nn.Sequential(
          nn.Linear(self.hidden_dim, n_classes),
          nn.Softmax()
      )
    def forward(self, x):
      x = self.embed(x)
      h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
      x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
      # x, _ = self.rnn(x,h_0)
      h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
      self.dropout(h_t)
      logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
      return logit

    def _init_state(self, batch_size=1):
      weight = next(self.parameters()).data
      return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [143]:
net = GRU(1, 256, n_vocab, embedd_size, output_size, 0.5).to(device)

In [144]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr)

In [136]:
losses = []
for epoch in range(epochs):
  
  for x, y in dataloader:
    optimizer.zero_grad()
    y = y.to(device)

    # forward 연산
    hypothesis = net(x)

    # 비용 함수
    y = y.squeeze()
    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()
    losses.append(cost.item()) # 값만 가져오기 위해서 .item()

  # 10의 배수에 해당되는 에포크마다 비용을 출력
  if epoch % 10 == 0:
      print(epoch, cost.item())

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0 1.0998486280441284
10 1.1001875400543213
20 1.103826880455017
30 1.102325201034546


In [146]:
dataset = CustomDataset_test()
test_loader = DataLoader(dataset, batch_size=batch_size)

In [138]:
correct = 0

with torch.no_grad():
  net = net.to('cpu')
  net.eval()
  for data, target in test_loader:
    data, target = data, target
    output = net(data)
    
    pred = output.max(1, keepdim=True)[1]
    # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
    correct += pred.eq(target.view_as(pred)).sum().item()

test_accuracy = correct / len(test_loader.dataset)
print('Accuracy:', test_accuracy)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Accuracy: 0.3432
